# Manejo de Datos en PyTorch

## Librerías

In [ ]:
import csv
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import json
import tempfile
import gzip
import json

from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from torch.utils.data import Dataset, DataLoader, IterableDataset

## La clase Dataset

La clase abstracta [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) es la clase base para construir un dataset de PyTorch. Cualquier dataset personalizado debe heredar de dicha clase e implementar los siguientes métodos:

- `__len__`: Para que `len(dataset)` devuelva el tamaño del conjunto de datos.
- `__getitem__`: Para soportar indexado de manera que `dataset[i]` devuelva el elemento `i`. Es común que en ciertos casos se utilice este método para levantar el dato real (e.g. una imagen) mientras que lo que se guarde en el dataset sea sólo una referencia a dicho dato (e.g. un path a la imagen). De esta manera se evita cargar muchas imágenes en memoria, haciendo que sea menos demandante a nivel RAM.

In [ ]:
class IMDBReviewsDataset(Dataset):
    def __init__(self, path, transform=None):
        self.dataset = pd.read_csv(path)
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.tolist()  # Deal with list of items instead of tensor
        
        item = {
            "data": self.dataset.iloc[item]["review"],
            "target": self.dataset.iloc[item]["sentiment"]
        }

        if self.transform:
            item = self.transform(item)
        
        return item

dataset = IMDBReviewsDataset("./data/imdb_reviews.csv.gz")
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

## Transformaciones

El ejemplo anterior nos muestra el uso básico, pero claramente no podemos pasarle eso a una red neuronal, no puede manejar texto. Es para eso que tenemos que hacer algún tipo de transformación sobre los atributos (en este caso el único atributo es el texto). 

### Normalización

En particular, como vemos en el caso anterior, el texto no está normalizado, parte de las transformaciones pueden incluir realizar algún tipo de normalización. Para eso hagamos uso de [`gensim`](https://radimrehurek.com/gensim/index.html), en particular utilizaremos el módulo [`preprocessing`](https://radimrehurek.com/gensim/parsing/preprocessing.html#module-gensim.parsing.preprocessing) que se encargará de hacer varias normalizaciones por defecto

In [ ]:
class TextPreprocess:
    def __init__(self, filters=None):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _encode_target(self, target):
        return 1 if target == "positive" else 0

    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self._preprocess_string(item["data"])
        else:
            data = [self._preprocess_string(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self._encode_target(item["target"])
        else:
            target = [self._encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

preprocess = TextPreprocess()
print(preprocess(dataset[0]))

### Conversión a vectores

Podemos continuar convertiendo el texto en una representación por vectores. Si bien hay muchas posibilidades (siendo la bolsa de palabras una de las más utilizadas), en general para Deep Learning se prefieren representaciones utilizando vectores contínuos, obtenidos por algún método del estilo de Word2Vec, Glove o FastText. Para este caso utilizaremos las representaciones de Glove de dimensión 50 que se dejaron para descargar en el [notebook 0](./0_set_up.ipynb).

In [ ]:
class VectorizeText:
    def __init__(self, glove_vectors_path):
        self.glove_model = KeyedVectors.load_word2vec_format(glove_vectors_path,
                                                             binary=False, no_header=True)
        self.unkown_vector = np.random.randn(self.glove_model.vector_size)  # Random vector for unknown words
    
    def _get_vector(self, word):
        if word in self.glove_model:
            return self.glove_model[word]
        else:
            return self.unkown_vector
    
    def _get_vectors(self, sentence):
        return np.vstack([self._get_vector(word) for word in sentence])
    
    def __call__(self, item):
        review = []
        if isinstance(item["data"][0], str):
            review = self._get_vectors(item["data"])
        else:
            review = [self._get_vectors(d) for d in item["data"]]

        return {
            "data": review,
            "target": item["target"]
        }

vectorizer = VectorizeText("./data/glove.6B.50d.txt.gz")
vectorizer(preprocess(dataset[0]))

### Combinación de vectores

Si bien ahora estamos con una versión de los atributos que podría pasar por una red neuronal, hay un problema, las distintas reviews tienen largo distinto y como el algoritmo se entrena en lotes (*mini-batches*) estas requieren tener todas el mismo largo. Hay varias maneras de lidiar con esto, cada una con sus ventajas y desventajas. Dado que por ahora solo vimos perceptrón multicapa, que espera algo de tamaño fijo, una opción sencilla puede ser la de simplemente promediar los vectores de palabras.

In [ ]:
class WordVectorsAverage:
    def __call__(self, item):
        if item["data"][0].ndim == 2:
            data = np.vstack([np.mean(d, axis=0) for d in item["data"]])
        else:
            data = np.mean(item["data"], axis=0)
        
        return {
            "data": data,
            "target": item["target"]
        }

vector_average = WordVectorsAverage()
vector_average(vectorizer(preprocess(dataset[0])))

### Conversión de vectores a tensores

En el paso final, debemos convertir nuestros datos de arrays de `numpy` a tensores de PyTorch.

In [ ]:
class ToTensor:
    def __call__(self, item):
        """
        This espects a single array.
        """
        return {
            "data": torch.from_numpy(item["data"]),
            "target": torch.tensor(item["target"])
        }

to_tensor = ToTensor()
to_tensor(vector_average(vectorizer(preprocess(dataset[0]))))

### Componiendo las transformaciones

Para evitar tener que llamar a todas las funciones de transformación que querramos aplicar, para ello hacemos uso del parámetro `transform` que definimos en nuestro `Dataset` y un poco de ayuda de `functools`.

In [ ]:
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions, lambda x: x)

dataset = IMDBReviewsDataset("./data/imdb_reviews.csv.gz",
                             transform=compose(preprocess, vectorizer, vector_average, to_tensor))
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

### Iterando el dataset

Ya tenemos nuestro conjunto de datos con sus respectivas transformaciones. ¿Para qué nos sirve esto? Una opción es simplemente iterar en el conjunto de datos de a un elemento. Esto es sencillo, simplemente se hace a través de un `for`.

In [ ]:
for idx, sample in enumerate(dataset):
    print(sample["data"])
    print(sample["target"])
    print("=" * 50)
    
    if idx == 2:
        break

## La clase Dataloader

El problema con iterar de a un elemento es que estamos limitados al querer entrenar un modelo. Por empezar, los modelos de Deep Learning suelen ser más eficientes si se entrenan utilizando algún tipo de entrenamiento por *mini-batches*. Además, hay otras cosas como mezclar los elementos (*shuffling*) o cargar datos en paralelo vía distintos *multiprocess workers*. La clase [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) precisamente se encarga de hacer eso por nosotros:

In [ ]:
dataloader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2
)

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, 
          sample_batched['data'].size(),
          sample_batched['target'].size())

    if i_batch == 2:
        print(sample_batched["data"])
        print(sample_batched["target"])
        break

## La clase IterableDataset

El método preferido para trabajar con conjuntos de datos en PyTorch es `torch.utils.data.Dataset`. En general, hacer uso inteligente del método `__getitem__`, e.g. usándolo para cargar imágenes a medida que sean necesitadas y no al instanciar el dataset, es la mejor manera de trabajar con un conjunto de datos. En particular, de esta forma es mucho más fácil hacer *shuffling* de los datos y demás. No obstante, no siempre esto es posible, muchas veces el conjunto de datos es demasiado grande para levantarlo en memoria (aunque sólo levantemos referencias). Para esos casos, PyTorch ofrece la clase [`torch.utils.data.IterableDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset), en este caso el único método que es requerido implementar es `__iter__`.

In [ ]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None):
        self.dataset_path = path
        self.transform = transform

    def __iter__(self):
        with gzip.open(self.dataset_path, "rt") as fh:
            for l in fh:
                data = json.loads(l)
                item = {
                    "data": data['title'],
                    "target": data['category']
                }
                
                if self.transform:
                    yield self.transform(item)
                else:
                    yield item

dataset = MeLiChallengeDataset("./data/meli-challenge-2019/spanish.train.jsonl.gz")
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0)
dataiter = iter(dataloader)
print(f"Sample batch:\n{dataiter.next()}")